In [1]:
#!pip install -r requirements.txt -q

In [2]:
import gmpy2
import numpy as np
from preprocessing import *
from sage.libs.pari.all import pari
from sage.all import euler_phi, gcd, Zmod, cyclotomic_polynomial, PolynomialRing

SEED = 777
rng = np.random.default_rng(SEED)

pari.allocatemem(2 * 1024**3)

PARI stack size set to 2147483648 bytes, maximum size set to 2147483648


Generar el algebra $A_q$ dado el campo $p^k$ y el tamano del mensaje

In [ ]:
n = 3
# 2**64 + 13
p = gmpy2.mpz(2)**16 + 1  # primo que define el cuerpo de base 𝔽_p
k = 1   # queremos incluir 𝔽_{p^k} dentro de A
# p = 2
# k = 8
s = 3   # número de copias de 𝔽_{p^k} que necesitamos

tau = p/2
sec = 40
delta = 1.0052

min_m = 14620
max_m = 20000  # rango de búsqueda de índices ciclotómicos m
m, phi_m, coeffs = calculate_m(p, k, s, min_m, max_m)

# m = 17425
# phi_m = int(euler_phi(m))

# Phi_ZZ = cyclotomic_polynomial(m)
# R = PolynomialRing(Zmod(p), 'x')
# Phi = R(Phi_ZZ)    
# coeffs = [int(c) for c in Phi.list()]
N = phi_m
# print("(m, N)=", (m, N))

c_sec = 9 * np.power(float(N), 2) * np.power(float(sec), 4) * np.power(2.0, sec + 8)


m = 14701 phi(m) = 14400 degrees: [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]
coeff

In [4]:
factors = unique_prime_factors(m)
if len(factors) == 1:
    C_m = 4/np.pi
elif np.prod(factors) <= 400:
    C_m = 8.6
else:
    # Ejecutar en paralelo (aumenta trials para mayor precisión)
    C_m = float(calculate_Cm_parallel(m, total_trials=1000))

    print(f"\nResultado Monte Carlo Paralelo:")
    print(f"C_m estimado: {C_m:.2f}")

    # Margen de seguridad (x1.5 o x2 es común en la práctica)
    margin = 1
    cm_seguro = C_m * margin

    print(f"Valor recomendado (con margen x{margin}): {cm_seguro:.2f}")

--- Iniciando Monte Carlo Paralelo (L2 Norm) para m=14701 ---
--- Repartiendo 1000 pruebas en 12 núcleos ---

Resultado Monte Carlo Paralelo:
C_m estimado: 15.52
Valor recomendado (con margen x1): 15.52


## 💡 El Problema Central: Dependencia Circular

Imagina que estás tratando de ajustar dos perillas, $q$ (tamaño del contenedor) y $r$ (cantidad de ruido), pero están conectadas por una cuerda elástica:
1. Regla de Corrección: Si subes el ruido ($r$), necesitas un contenedor ($q$) más grande para que los datos quepan.
    - Causa: A mayor $r \to$ mayor $q$
2. Regla de Seguridad: Si agrandas el contenedor ($q$), el problema matemático se vuelve "más fácil" de romper, por lo que necesitas subir el ruido ($r$) para esconder el secreto de nuevo.
    - Causa: A mayor $q \to$ mayor $r$.

Esto crea un bucle: Subir $r$ te obliga a subir $q$, lo cual te obliga a subir $r$ de nuevo...El algoritmo busca el punto exacto donde dejas de necesitar subir ambos valores. Ese es el "Punto Fijo".

In [5]:
# Valor inicial semilla para r (rho)
r_seed = 3.2 

# Ejecutar el método numérico
q, r = fixed_point_qr(r_seed, N, C_m, p, tau, c_sec, n, sec, delta)

print("\n=============================================")
print("      RESULTADOS DE PARÁMETROS OPTIMIZADOS")
print("=============================================")
print(f"1. Módulo (q)        : {int(q):.4e}")
print(f"   En potencias de 2 : 2^{gmpy2.log2(q):.2f}")
print("---------------------------------------------")
print(f"2. Ruido std (r/rho) : {r:.6f}")
print(f"   (Debe ser >= 3.2) : {'CUMPLE' if r >= 3.2 else 'FALLA'}")
print("=============================================")

--- Iniciando Iteración (N=14400, sec=40) ---
✅ Convergencia en iteración 1

      RESULTADOS DE PARÁMETROS OPTIMIZADOS
1. Módulo (q)        : 3.9402e+115
   En potencias de 2 : 2^384.00
---------------------------------------------
2. Ruido std (r/rho) : 3.200000
   (Debe ser >= 3.2) : CUMPLE


## Generate keys

In [6]:
Aq = generate_Aq(q, coeffs)

# Key Generation
a = Aq.random_element()
s_coeff, e_coeff= sample_discrete_gaussian_ZN(N, s, q, rng, num_samples=2)
sk = Aq(s_coeff.tolist())
e = Aq(e_coeff.tolist())
b = (a*sk) + (p*e)

pk = (a, b)
# sk

# Key hat Generation
a_hat = Aq.random_element()
b_hat = Aq.random_element()

pk_hat = (a_hat, b_hat)

Construido A_q = Univariate Quotient Polynomial Ring in a over Ring of integers modulo 39402006196394479212279040100143613805079739270465446667948293404245721771497210611414266254884915640806627990306816 with modulus x^14400 + 65536*x^14399 + x^14339 + 65536*x^14338 + x^14278 + 65536*x^14277 + x^14217 + 65536*x^14216 + x^14159 + 65536*x^14158 + x^14156 + 65536*x^14155 + x^14098 + 65536*x^14097 + x^14095 + 65536*x^14094 + x^14037 + 65536*x^14036 + x^14034 + 65536*x^14033 + x^13976 + 65536*x^13975 + x^13973 + 65536*x^13972 + x^13918 + 65536*x^13917 + x^13915 + 65536*x^13914 + x^13912 + 65536*x^13911 + x^13857 + 65536*x^13856 + x^13854 + 65536*x^13853 + x^13851 + 65536*x^13850 + x^13796 + 65536*x^13795 + x^13793 + 65536*x^13792 + x^13790 + 65536*x^13789 + x^13735 + 65536*x^13734 + x^13732 + 65536*x^13731 + x^13729 + 65536*x^13728 + x^13677 + 65536*x^13676 + x^13674 + 65536*x^13673 + x^13671 + 65536*x^13670 + x^13668 + 65536*x^13667 + x^13616 + 65536*x^13615 + x^13613 + 65536*x^13612 + x^1

## Encode/Decode messages

In [ ]:
print("\n--- Iniciando Prueba de Encoding/Decoding ---")

# 1. Preparar la estructura SIMD
R_p, slots_moduli = prepare_slots(p, coeffs)
num_slots = len(slots_moduli)
print(f"Sistema configurado con {num_slots} slots disponibles.")

# 2. Definir mensajes de prueba
# Solo usaremos 4 valores, el resto serán ceros
mis_mensajes = [ord("H"), ord("o"), ord("l"), ord("a")] 
#mis_mensajes = [13,14,15,16] 

print(f"Mensajes originales: {mis_mensajes}")

# 3. Codificar (Encode)
plaintext_poly = encode(mis_mensajes, R_p, slots_moduli, Aq)

print("Codificación exitosa. Elemento en Aq creado.")
# print(plaintext_poly) # Descomentar para ver el polinomio gigante

# 4. Decodificar (Decode) para verificar
mensajes_recuperados = decode(plaintext_poly, R_p, slots_moduli)

# Filtramos solo los primeros 4 para comparar visualmente
print(f"Mensajes recuperados: {mensajes_recuperados[:4]}")

# Verificación automática
assert mensajes_recuperados[:4] == mis_mensajes
print("¡Éxito! Los mensajes coinciden.")